In [ ]:
!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

1.9.0+cu102 True


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)


model_final_f10217.pkl: 178MB [00:06, 25.8MB/s]                           
The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd

In [ ]:
def crop(img, box):
  x0=int(box[0])
  y0=int(box[1])
  x1=int(box[2])
  y1=int(box[3])
  #print(x0,y0,x1,y1)
  crop_img = img[y0:y1, x0:x1]
  return crop_img

In [ ]:
 folder='/foldername'
 #folder_list=os.listdir(folder)
 results_folder='/cfoldername'

 #folder_list=pd.read_csv("/csvname")
 counter=18993
 for file in folder_list["0"][18993:]:
  print(counter)
  counter+=1
  path=os.path.join(folder,file)
  print(path)
  im = cv2.imread(path)
  outputs = predictor(im)
  temp_count=0
  mask_array = outputs['instances'].pred_masks.to("cpu").numpy()
  num_instances = mask_array.shape[0]
  scores = outputs['instances'].scores.to("cpu").numpy()
  labels = outputs['instances'].pred_classes .to("cpu").numpy()
  bbox   = outputs['instances'].pred_boxes.to("cpu").tensor.numpy()
  human=np.where(labels==0)[0].tolist() # extracting only humans
  mask_array = np.moveaxis(mask_array, 0, -1)
  mask_array_instance = []

  for i in range(1): # getting only the firss >> depend on the data set
      temp_img=np.copy(im)
      img = np.zeros_like(im)
      mask_array_instance.append(mask_array[:, :, i:(i+1)])
      if i in human:
        temp_img= np.where(mask_array_instance[i] == False, 255, temp_img)
        res_img=crop(temp_img,bbox[i])
        result_name=file
        result=os.path.join(results_folder,result_name)
        cv2.imwrite(result,res_img)
        #cv2_imshow(temp_img)
        #cv2_imshow(res_img)
        temp_count+=1
      else:
        continue
      
  #img_mask = np.asarray(img_mask)
  #output = cv2.addWeighted(im, 0.7, img_mask, 0.3, 0) #blending 2 images